In [1]:
%load_ext autoreload
%autoreload 2
import torch
import json
import os
import argparse
from rwkv_model import RUN_CUDA
from wkv_kernel import WKV, WKVConfig

/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


In [2]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [3]:
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

### Compare BlinkDL CUDA kernel with own embedded version

In [4]:
# input tensors
batch_size = 2
seq_len = 5
embedding_dim = 7
attention_dim = 7
time_decay = torch.randn((attention_dim,))
time_first = torch.randn((attention_dim,))
k = torch.randn((batch_size, seq_len, embedding_dim))
v = torch.randn((batch_size, seq_len, embedding_dim))


In [5]:
out_ground_truth = RUN_CUDA(batch_size, seq_len, embedding_dim, time_decay, time_first, k, v)

In [6]:
out_ground_truth

tensor([[[-0.4895, -0.3953, -0.5015, -0.1135,  1.5791,  0.3691,  0.4893],
         [ 0.2627, -0.0813, -0.1442,  0.2314, -0.6914, -0.8613, -0.0439],
         [ 1.3896, -0.0635,  0.6626, -0.4768, -0.5508,  0.2734, -1.8936],
         [ 1.4307, -0.4177,  0.8262, -0.3821,  0.2301,  0.7666, -0.7812],
         [ 0.8364, -0.5107,  0.6519,  0.1125, -1.4043, -1.3096,  1.2422]],

        [[-0.8867, -0.5200,  1.7021, -1.0928, -2.6758,  0.1827, -0.3997],
         [-1.7266, -0.5405,  0.5117, -0.9746, -1.5459, -1.2070,  0.6060],
         [-1.6982, -0.3269,  0.2285,  0.6382, -0.2129, -1.1562,  1.2676],
         [-1.1133,  0.3591,  0.2654,  0.5171,  0.0053,  0.0396,  0.8882],
         [-0.5884,  0.2141,  0.4832,  0.4839,  0.8008,  0.0890,  0.4238]]],
       device='cuda:0', dtype=torch.float16)

In [7]:
wkv = WKV(config=WKVConfig(float_mode='fp16'))

Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module wkv, skipping build step...
Loading extension module wkv...


In [8]:
out = wkv(batch_size, seq_len, embedding_dim, time_decay, time_first, k, v)

In [9]:
out

tensor([[[-0.4895, -0.3953, -0.5015, -0.1135,  1.5791,  0.3691,  0.4893],
         [ 0.2627, -0.0813, -0.1442,  0.2314, -0.6914, -0.8613, -0.0439],
         [ 1.3896, -0.0635,  0.6626, -0.4768, -0.5508,  0.2734, -1.8936],
         [ 1.4307, -0.4177,  0.8262, -0.3821,  0.2301,  0.7666, -0.7812],
         [ 0.8364, -0.5107,  0.6519,  0.1125, -1.4043, -1.3096,  1.2422]],

        [[-0.8867, -0.5200,  1.7021, -1.0928, -2.6758,  0.1827, -0.3997],
         [-1.7266, -0.5405,  0.5117, -0.9746, -1.5459, -1.2070,  0.6060],
         [-1.6982, -0.3269,  0.2285,  0.6382, -0.2129, -1.1562,  1.2676],
         [-1.1133,  0.3591,  0.2654,  0.5171,  0.0053,  0.0396,  0.8882],
         [-0.5884,  0.2141,  0.4832,  0.4839,  0.8008,  0.0890,  0.4238]]],
       device='cuda:0', dtype=torch.float16)

In [ ]:
# the outputtensors are the same! -> NO BUG